In [2]:
import pathlib

import matplotlib as mpl
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.ticker import AutoMinorLocator, FormatStrFormatter, MultipleLocator

savefig_args = {
    "dpi": 300,
    "bbox_inches": "tight",
    "pad_inches": 0,
    "transparent": True,
}
output_dir = "../../figures/figure2"
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
output_suffix = ""
output_formats = [".svg", ".png", ".eps"]
filetype = "pdf"
sc.settings.figdir = output_dir
sc.set_figure_params(format="pdf", transparent=True)
sc.set_figure_params(dpi_save=600, figsize=(3.5, 3.5))


def save_figure(
    fig,
    name,
    output_dir=output_dir,
    output_suffix=output_suffix,
    output_formats=output_formats,
    savefig_args=savefig_args,
):
    for output_format in output_formats:
        fig.savefig(
            output_dir + "/" + name + output_suffix + output_format, **savefig_args
        )
    return None


width = plt.rcParams["figure.figsize"][0]
height = plt.rcParams["figure.figsize"][1]

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 100)
plt.style.use('seaborn-whitegrid')
plt.style.use("../../scripts/paper.mplstyle")
%run ../../scripts/plotting_helper.py

In [3]:
bcells = sc.read_h5ad("../../processed_data/h5ad_objects/bcells.h5ad")

In [ ]:
# dropnans
bcells = bcells[
    bcells.obs.dropna(subset=["simple_mutation_status", "isotype_simple"]).index
]

In [ ]:
# variables
x = "fraction_mutated_bases"
data = bcells.obs
hue = "bcelltype"
palette = bcelltype_colors
# Plot
fig, ax = plt.subplots(1, 1)
sns.ecdfplot(data=data, x=x, complementary=True, hue=hue, lw=1.5, palette=palette)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0)
plt.vlines(mutation_cutoffs[0], ymin=0, ymax=1, linestyles="-", color="grey", linewidths=3)
plt.vlines(mutation_cutoffs[1], ymin=0, ymax=1, linestyles="-.", color="grey", linewidth=3)
plt.xlabel("Fraction of V-gene bases mutated")
plt.ylabel("Proportion of Cells")
# TODO add legend for the cutoffs
save_figure(fig, "hypermutation_cutoffs")

# Metrics Classification

In [ ]:
label1 = "bcelltype"
label2 = "simple_mutation_status"
data = bcells.obs
# calculate confusion matrix
cmtx = sc.metrics.confusion_matrix(label1, label2, data, normalize=True)
# Plot
kws = dict(cbar_kws=dict(ticks=[0, 0.50, 1], orientation="horizontal"))
g = sns.clustermap(cmtx, cmap="viridis", figsize=(width, height),**kws)
x0, _y0, _w, _h = g.cbar_pos
g.ax_cbar.set_position([0.7, 0.09, g.ax_row_dendrogram.get_position().width * 3, 0.05])
g.ax_cbar.set_title('proportion of cells')
g.ax_cbar.tick_params(axis="x", length=10)
#g.ax_cbar.label("Proportion of Cells")
for spine in g.ax_cbar.spines:
    g.ax_cbar.spines[spine].set_color("k")
    g.ax_cbar.spines[spine].set_linewidth(1)
fig = g.figure
save_figure(fig, "cmtx_{}_{}".format(label1, label2))

In [ ]:
label1 = "bcelltype"
label2 = "isotype_simple"
data = bcells.obs
cmtx = sc.metrics.confusion_matrix(label1, label2, data, normalize=True)
g = sns.clustermap(cmtx, cmap="viridis")
fig = g.figure
save_figure(fig, "cmtx_{}_{}".format(label1, label2))

# Bias Calculation

# Groupby mutation status

In [ ]:
ird = bcells.obs

In [ ]:
# subset to activated B cells
data = ird[ird.sample_id.str.contains("Day")]
data = data[~data.sample_id.str.contains("Day 0")]
group = "mutation_status"
label = "bcelltype"
palette = bcelltype_colors
data = data.groupby(group)[label].value_counts(normalize=False).reset_index()
data.columns = [group, label, "proportion"]

fig, ax = plt.subplots(1, 1)
sns.barplot(data=data, y=group, x="proportion", hue=label, palette=palette)
plt.xscale("log")
plt.xlabel("Number of Cells")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0)

In [ ]:
import scipy

# BarPlot
adata = bcells
df = pd.DataFrame(
    adata.obs.groupby("sample_id").mutation_status.value_counts(normalize=True)
).reset_index()
data = df.pivot_table(values="mutation_status", index=df.sample_id, columns="level_1")
# set seaborn plotting aesthetics
# subset to timecourse & put in order
res_arr = np.flip(pseudo_timecourse)
color = mutation_colors
data = data.loc[res_arr]

ax = data.plot(kind="barh", stacked=True, color=color)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0)
locs, labels = plt.xticks()
plt.xlabel("Proportion of Cells")
plt.setp(labels, rotation=0)
fig = ax.get_figure()
save_figure(fig, "stacked_bar_mutation_dynamics")

## Ratio Plot

In [ ]:
ird["sample_id"] = ird["sample_id"].cat.remove_unused_categories()

In [ ]:
group = "sample_id"
label = "mutation_status"
df, original = bootstrap_routine(ird, "sample_id", "mutation_status", 1000, 1000)

In [ ]:
group = "sample_id"
data = df
y = "ratio"
x = "sample_id"
hue = "mutation_status"
order = ["Day 0", "Day 4", "Day 8", "Day 12", "BM CD138+"]
scale = 1
linestyles = ["-.", ":", "--"]
fig, ax = plt.subplots(1, 1)
sns.pointplot(
    data=data,
    x=x,
    y=y,
    ci=None,
    join=False,
    hue=hue,
    markers=["_", "_", "_"],
    estimator=lambda x: np.percentile(x, 97.5),
    alpha=0.5,
    legend=False,
    scale=scale,
    order=order,
    palette=mutation_colors,
    dodge=0.3,
)

sns.pointplot(
    data=data,
    x=x,
    y=y,
    ci=None,
    join=False,
    hue=hue,
    estimator=lambda x: np.percentile(x, 2.5),
    alpha=0.5,
    scale=scale,
    markers=["_", "_", "_"],
    order=order,
    palette=mutation_colors,
    legend=False,
    dodge=0.3,
)

sns.pointplot(
    data=data,
    x=x,
    y=y,
    errorbar="pi",
    join=True,
    linestyles=linestyles,
    hue=hue,
    linewidth=0.1,
    order=order,
    palette=mutation_colors,
    scale=scale,
    markers=["o", "*", "^"],
    dodge=0.3,
)
plt.axhline(y=1, xmin=0, xmax=1, ls="-.", color="k", lw=2, alpha=0.6)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0)
plt.yscale("log", base=2)
plt.ylabel("Relative Abundance\n(cf. Day 0)")
plt.xlabel("Sample")
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)

In [ ]:
save_figure(fig, "PopluationDynamicsMutationBucket")

# Isotype Population Dynamics

In [ ]:
def calculation_routine_3(df, group1, group2, group3):
    dfs = []
    for index, group in df.groupby([group1, group2]):
        stats = pd.DataFrame(
            df.groupby([group1, group2])[group3]
            .value_counts(normalize=True)
            .xs(index[0])
            .xs("mutated")
            / df.groupby([group1, group2])[group3]
            .value_counts(normalize=True)
            .xs(index[0])
            .xs("germline")
        )
        stats["sample_id"] = index[0]
        dfs.append(stats)
    # Create DataFrame to Plot
    data = pd.concat(dfs)
    data.columns = ["germline / mutated", "sample_id"]
    data.index.rename(group3, inplace=True)
    data.reset_index(inplace=True)
    data.drop_duplicates(inplace=True)
    return data, stats

### Generate Bootstrapped Samples

In [ ]:
dfs = []
for i in range(1000):
    # create bootstrapped df
    df = ird.groupby("sample_id").sample(frac=1, replace=True)
    data = (
        df.groupby(["sample_id", "isotype_simple"])
        .simple_mutation_status.value_counts(normalize=True)
        .unstack()
        .reset_index()
    )
    data["germline / mutated"] = data["germline"] / (data["mutated"] + 0)
    dfs.append(data)
data = pd.concat(dfs)


real = (
    ird.groupby(["sample_id", "isotype_simple"])
    .simple_mutation_status.value_counts(normalize=True)
    .unstack()
    .reset_index()
)
real.loc[:, "germline / mutated"] = real["germline"] / real["mutated"]


maximum = data["germline / mutated"][data["germline / mutated"] != np.infty].max()

data

# data.loc[data['germline / mutated'] >= data['germline / mutated'].max(), 'germline / mutated'] = maximum

order = ["Day 0", "Day 4", "Day 8", "Day 12", "BM CD138+"]

mean = data.groupby(["sample_id", "isotype_simple"]).mean().reset_index()
q1 = data.groupby(["sample_id", "isotype_simple"]).quantile(q=0.975).reset_index()
q2 = data.groupby(["sample_id", "isotype_simple"]).quantile(q=0.025).reset_index()

markers = []
linestyles = []
for i in range(len(order)):
    markers.append("_")
    linestyles.append("--")

In [ ]:
palette = igh_colors_simple

In [ ]:
data = (
    ird.groupby(["sample_id", "isotype_simple"])
    .simple_mutation_status.value_counts(normalize=True)
    .unstack()
    .reset_index()
)
data["germline / mutated"] = data["germline"] / (data["mutated"] + 0)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax = sns.pointplot(
    data=q1,
    y="germline / mutated",
    x="sample_id",
    hue="isotype_simple",
    size=1,
    order=order, join = True,
    scale=0.5,
    linestyles=linestyles,
    markers=markers,
    palette=palette,
)
ax.legend_.remove()
ax = sns.pointplot(
    data=q2,
    y="germline / mutated",
    x="sample_id",
    hue="isotype_simple",
    size=0.5,
    order=order,
    scale=0.5, join = True,
    linestyles=linestyles,
    markers=markers,
    palette=palette,
)
ax.legend_.remove()
ax = sns.pointplot(
    data=data,
    y="germline / mutated",
    x="sample_id",
    hue="isotype_simple",
    dashes=False, join = True,
    order=order,
    scale=1,
    palette=palette,
)

plt.legend(bbox_to_anchor=(1.1, 1), loc="upper left", borderaxespad=0)
plt.ylabel("Ratio Germline / Mutated")
plt.yscale("log", base=2)
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)

In [ ]:
save_figure(fig, "germline_over_mutated_class_switching")

# Celltype Population Dynamics

In [ ]:
dfs = []
for i in range(1000):
    # create bootstrapped df
    df = ird.groupby("sample_id").sample(frac=1, replace=True)
    data = (
        df.groupby(["sample_id", "bcelltype"])
        .simple_mutation_status.value_counts(normalize=True)
        .unstack()
        .reset_index()
    )
    data["germline / mutated"] = data["germline"] / data["mutated"]
    dfs.append(data)

data = pd.concat(dfs)


real = (
    ird.groupby(["sample_id", "bcelltype"])
    .simple_mutation_status.value_counts(normalize=True)
    .unstack()
    .reset_index()
)
real.loc[:, "germline / mutated"] = real["germline"] / real["mutated"]


maximum = data["germline / mutated"][data["germline / mutated"] != np.infty].max()

data

# data.loc[data['germline / mutated'] >= data['germline / mutated'].max(), 'germline / mutated'] = maximum

order = ["Day 0", "Day 4", "Day 8", "Day 12", "BM CD138+"]

mean = data.groupby(["sample_id", "bcelltype"]).mean().reset_index()
q1 = data.groupby(["sample_id", "bcelltype"]).quantile(q=0.975).reset_index()
q2 = data.groupby(["sample_id", "bcelltype"]).quantile(q=0.025).reset_index()

markers = []
linestyles = []
marknumber = mean["bcelltype"].nunique()
for i in range(marknumber):
    markers.append("_")
    linestyles.append("--")

In [ ]:
order = pseudo_timecourse

In [ ]:
palette = bcelltype_colors

In [ ]:
fig, ax = plt.subplots(1, 1)
scale = 0.6
sns.pointplot(
    data=mean,
    y="germline / mutated",
    x="sample_id",
    hue="bcelltype",
    dashes=True,
    order=order,
    scale=0.8,
    palette=palette,
)
sns.pointplot(
    data=q1,
    y="germline / mutated",
    x="sample_id",
    hue="bcelltype",
    size=1,
    order=order,
    scale=scale * 0.3,
    markers=markers,
    linestyles=linestyles,
    palette=palette,
)
sns.pointplot(
    data=q2,
    y="germline / mutated",
    x="sample_id",
    hue="bcelltype",
    size=1,
    order=order,
    scale=scale * 0.3,
    markers=markers,
    linestyles=linestyles,
    palette=palette,
)
plt.legend(bbox_to_anchor=(1.1, 1), loc="upper left", borderaxespad=0)
plt.ylabel("Ratio Germline / Mutated")
plt.yscale("log", base=2)
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)
sns.move_legend(
    ax,  loc="upper left",bbox_to_anchor=(1, 1),
    ncol=3, title=None, frameon=False,
)

In [ ]:
save_figure(fig, "B_Cell_Subtypes_Dynamics_Mutation")

In [ ]:
group = "sample_id"
label = "bcelltype"
order = pseudo_timecourse
_df = pd.DataFrame(ird.groupby([group])[label].value_counts(normalize=False))
_df.columns = ["dummy"]
_df.reset_index(inplace=True)
_df.columns = ["sample_id", "celltype", "proportion"]
fig, ax = plt.subplots(1, 1, figsize=(5, 2))

sns.barplot(
    x="sample_id",
    y="proportion",
    hue="celltype",
    palette=palette,
    data=_df,
    order=order,
)

# plt.xscale('log')
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", borderaxespad=0)
plt.xlabel("Number of Cells")
plt.ylabel("sample_id")
plt.yscale("log")
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)

In [ ]:
save_figure(fig, "bcelltype_count")

In [ ]:
ird.loc[:, "Fraction V-gene mutated"] = 1 - ird["IR_VDJ_1_v_identity"]

In [ ]:
ird = ird[ird.sample_id.isin(pseudo_timecourse)]

In [ ]:
ird.sample_id = ird.sample_id.cat.remove_unused_categories()

In [ ]:
g = sns.displot(
    ird,
    x="fraction_mutated_bases",
    hue="isotype_simple",
    kind="ecdf",
    col="sample_id",
    lw=3,
    palette=IGH_colors_simple(),
    complementary=True
)
g.set_ylabels("Proportion of Cells")

fig = g.figure

save_figure(fig, "ecdf_v_mutation_isotype_simple")

In [ ]:
g = sns.displot(
    ird,
    x="fraction_mutated_bases",
    hue="isotype_simple",
    kind="ecdf",
    col="sample_id",
    lw=3,
    palette=IGH_colors_simple(),
    complementary=True, stat = "count"
)
g.set_ylabels("Number of Cells")
plt.yscale("log")

fig = g.figure

save_figure(fig, "ecdf_v_mutation_isotype_simple_count")